<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/346_WDO_GapDetection_Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Workforce Development Orchestrator — Skill Gap Detection Node

The `skill_gap_detection_node` is responsible for transforming workforce data into a **ranked list of actionable skill gaps**. It is the point where raw information about skills, roles, and automation exposure becomes something leaders can actually act on.

Rather than inventing insight, this node **applies tested utilities deterministically** and enriches results with clear priority signals.

---

## 1. Explicit Preconditions Protect Decision Quality

The node begins by verifying that all required inputs are present:

* Employees
* Role definitions
* Skill definitions
* Existing skill gap data
* Automation risk analysis

If any of these are missing, execution stops immediately with a clear error message.

This prevents one of the most dangerous failure modes in analytics systems:

> Producing confident recommendations from incomplete data.

---

## 2. Organization-Wide Gap Detection Is Applied Consistently

The node delegates gap discovery to `detect_all_skill_gaps`, which applies the **same rules to every employee**.

This ensures:

* Fairness across the workforce
* Consistent logic regardless of department or role
* Results that can be compared and aggregated safely

No special cases. No hidden exceptions.

---

## 3. Automation Risk Is Embedded in the Analysis

Because automation risk analysis has already run, the node is able to **contextualize every skill gap**:

* Gaps in high-risk roles become more urgent
* Future skills matter more when automation pressure is high
* Low-risk roles are not over-prioritized prematurely

This sequencing is intentional and reflects real strategic reasoning:

> Risk should inform urgency — not the other way around.

---

## 4. Priority Scores Are Calculated Transparently

Each detected gap is assigned a **priority score** using configurable weights defined in the agent’s configuration.

This score:

* Balances automation risk, role criticality, future relevance, and urgency
* Is explainable and auditable
* Can be tuned without changing code

This avoids subjective ranking and creates a **policy-driven prioritization framework**.

---

## 5. Clean State Output for Downstream Decisions

The node returns a single, well-defined artifact:

* `skill_gap_analysis`

This structure is:

* Ready for sorting and filtering
* Suitable for dashboards and reports
* Designed for direct consumption by learning and prioritization nodes

No post-processing guesswork is required.

---

## 6. Failures Are Visible, Not Silent

Any unexpected error is captured and added to the shared error log rather than crashing the system.

This ensures:

* Partial execution can still be reviewed
* Root causes are preserved
* Trust is maintained even when things go wrong

---

## Why This Node Builds Executive Confidence

From a leadership perspective, this node guarantees that:

* Skill investments are justified by real risk
* Urgency is earned, not assumed
* Workforce planning decisions can be defended

From a system design perspective, it guarantees:

* Deterministic behavior
* Clean separation of concerns
* Predictable, testable outcomes

---

## Architectural Takeaway

This node demonstrates a core principle of responsible AI orchestration:

> **Prioritization should be policy-driven, not model-driven.**

By grounding skill gap urgency in explicit signals and configurable weights, the orchestrator produces recommendations leaders can trust — and employees can understand.



In [ ]:
def skill_gap_detection_node(
    state: WorkforceDevelopmentOrchestratorState,
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """
    Skill Gap Detection Node: Orchestrate detecting and analyzing skill gaps.

    Detects skill gaps for all employees, enhances them with automation risk context,
    and calculates priority scores.
    """
    errors = state.get("errors", [])

    # Get required data from state
    employees = state.get("employees", [])
    roles_lookup = state.get("roles_lookup", {})
    skills_lookup = state.get("skills_lookup", {})
    skill_gaps = state.get("skill_gaps", [])
    automation_risk_analysis = state.get("automation_risk_analysis", [])

    if not employees:
        return {
            "errors": errors + ["skill_gap_detection_node: employees data required"]
        }

    if not roles_lookup:
        return {
            "errors": errors + ["skill_gap_detection_node: roles_lookup data required"]
        }

    if not skills_lookup:
        return {
            "errors": errors + ["skill_gap_detection_node: skills_lookup data required"]
        }

    try:
        # Detect all skill gaps
        all_gaps = detect_all_skill_gaps(
            employees,
            roles_lookup,
            skills_lookup,
            skill_gaps,
            automation_risk_analysis
        )

        # Calculate priority scores for each gap
        enhanced_gaps = []
        for gap in all_gaps:
            gap["priority_score"] = calculate_gap_priority_score(gap, config)
            enhanced_gaps.append(gap)

        return {
            "skill_gap_analysis": enhanced_gaps,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"skill_gap_detection_node: Unexpected error: {str(e)}"]
        }



# Test skill gap detection node

In [ ]:
"""Test skill gap detection node

Testing Phase 4: Skill Gap Detection Node
Following the pattern: Test node after utilities pass
"""

from agents.workforce_development_orchestrator.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    automation_risk_analysis_node,
    skill_gap_detection_node
)
from config import (
    WorkforceDevelopmentOrchestratorState,
    WorkforceDevelopmentOrchestratorConfig
)


def test_skill_gap_detection_node():
    """Test skill gap detection node"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Load data and analyze automation risk first
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    # Then detect skill gaps
    result = skill_gap_detection_node(state, config)

    # Check that analysis is present
    assert "skill_gap_analysis" in result
    assert len(result["skill_gap_analysis"]) > 0

    # Check structure of first gap
    first_gap = result["skill_gap_analysis"][0]
    assert "employee_id" in first_gap
    assert "skill_id" in first_gap
    assert "gap_type" in first_gap
    assert "priority_score" in first_gap
    assert "automation_risk_context" in first_gap

    # Verify no errors
    assert len(result.get("errors", [])) == 0

    print("✅ test_skill_gap_detection_node: PASSED")


def test_skill_gap_detection_node_requires_data():
    """Test skill gap detection node requires data"""
    state: WorkforceDevelopmentOrchestratorState = {
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    result = skill_gap_detection_node(state, config)

    # Should have errors
    assert "errors" in result
    assert len(result["errors"]) > 0

    print("✅ test_skill_gap_detection_node_requires_data: PASSED")


def test_skill_gap_detection_integration():
    """Test skill gap detection with full workflow"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Full workflow up to skill gap detection
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    gap_update = skill_gap_detection_node(state, config)
    state.update(gap_update)

    # Verify all data is present
    assert "employees" in state
    assert "automation_risk_analysis" in state
    assert "skill_gap_analysis" in state
    assert len(state["skill_gap_analysis"]) > 0

    # Verify gap quality
    for gap in state["skill_gap_analysis"]:
        assert 0.0 <= gap["priority_score"] <= 1.0
        assert gap["gap_type"] in ["missing_required_skill", "missing_future_skill"]
        assert gap["automation_risk_context"] in ["high", "medium", "low", "unknown"]

    print("✅ test_skill_gap_detection_integration: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Skill Gap Detection Node (Phase 4)")
    print("=" * 60)
    print()

    test_skill_gap_detection_node()
    test_skill_gap_detection_node_requires_data()
    test_skill_gap_detection_integration()

    print()
    print("=" * 60)
    print("✅ All skill gap node tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_skill_gap_node.py
============================================================
Testing Skill Gap Detection Node (Phase 4)
============================================================

✅ test_skill_gap_detection_node: PASSED
✅ test_skill_gap_detection_node_requires_data: PASSED
✅ test_skill_gap_detection_integration: PASSED

============================================================
✅ All skill gap node tests passed!
============================================================
